# Stock index statistics

This notebook shows how to compute stock index statistics.

In [ ]:
import pandas as pd
import os
import pickle 

# load data
FILEPATH = "download_data_full/price_data/all_indexes_2006-01-01_2021-12-31.pickle"
with open(FILEPATH, "rb") as f:
    data = pickle.load(f)
    
#index_list = list(data.keys())

# define groups of indexes
indexes_usa = ['SPX','CCMP','RIY','RTY','RAY','RLV','RLG','NBI']
indexes_sp500 = ['S5COND','S5CONS','S5ENRS','S5FINL','S5HLTH','S5INFT',
                 'S5MATR','S5RLST','S5TELS','S5UTIL','S5INDU']
indexes_eu = ['DAX','CAC','UKX','BEL20','IBEX','KFX','OMX','SMI']
indexes_apac = ['AS51','HSI','STI']
indexes_jp = ['NKY','TPX']
indexes_bric = ['IBOV','RTSI$','NIFTY','MXIN','SHCOMP','SHSZ300']

# merge all groups into one
indexes_all = indexes_usa + indexes_sp500 + indexes_eu + indexes_apac + indexes_jp + indexes_bric

Loop over all indexes to compute empirical statistics (mean, median and mode). The resuts are saved into dictionary.

In [ ]:
from src.analyzer import StockIndexAnalyzer

start_date = "2006-12-29"
end_date = "2021-12-31"

results = {}

for index_name in indexes_all:
    
    print(f"Processing {index_name} index")
    
    stock_analyzer = StockIndexAnalyzer(prices = data[index_name],
                                        stock_index = index_name,
                                        start_date = start_date,
                                        end_date = end_date)
    print ("=== EXPERIMENTAL RESULTS ===")
    print ("Experimental mean, median, mode, mean/median, mean/mode: %.2f, %.2f, %.2f, %.2f, %.2f" %(stock_analyzer.mean_expt,
                                                                                          stock_analyzer.median_expt,
                                                                                          stock_analyzer.mode_expt,
                                                                                          stock_analyzer.mean_expt/stock_analyzer.median_expt,
                                                                                          stock_analyzer.mean_expt/stock_analyzer.mode_expt))
    results[index_name] = {'category': stock_analyzer.category,
                           'years': stock_analyzer.nyears,
                           'n_stocks': len(stock_analyzer.tickers),
                           'n_stocks_data': len(stock_analyzer.mu),
                           'mean': stock_analyzer.mean_expt,
                           'median': stock_analyzer.median_expt,
                           'mode': stock_analyzer.mode_expt,
                           'mean/median': stock_analyzer.mean_expt/stock_analyzer.median_expt,
                           'mean/mode': stock_analyzer.mean_expt/stock_analyzer.mode_expt}
    
    #break

Transform results dictionary into DataFrame. Transpose the table to have indexes as rows.

In [ ]:
df = pd.DataFrame.from_dict(results).T
df

Save dataframe with the results table into CSV file.

In [ ]:
import os 

DIR = 'results'
os.makedirs(DIR, exist_ok=True)

df.to_csv(DIR+f'/data_{stock_analyzer.nyears}.csv', header=True)